#Подготовка данных

Загрузим датасет.  
В датасете будут находиться данные по пробуренным скважинам.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML/DZ/Rec_Sys_MISIS.csv', 
                 #encoding = 'ISO-8859-1', 
                 index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data.head(8)

,Depth_m,Litology,Bit_load_tonnes,Rotor_speed,Inlet_density_g/cc,Inlet_flow_litres/sec,D_bit_mm,D_well_mm,Complication_code_mm,Stratigraphy_code,Markers_horizon_code,Oil_well,Aver_mech_speed_m/h
0,7.80,0,0.0,56,1.01,43.2682,393.7,324,102,20,NaN,6109,0.0690
1,7.90,0,0.0,56,1.01,42.9248,393.7,324,102,20,NaN,6109,4.7368
2,8.01,0,0.0,72,1.01,43.0965,393.7,324,102,20,NaN,6109,2.4444
3,8.11,0,0.0,72,1.01,43.0965,393.7,324,102,20,NaN,6109,7.3469
4,8.20,0,0.0,72,1.01,42.9248,393.7,324,102,20,NaN,6109,7.7143
5,8.30,0,0.0,72,1.01,43.0965,393.7,324,102,20,NaN,6109,13.3333
6,8.40,0,0.0,72,1.01,44.2984,393.7,324,102,20,NaN,6109,45.0000
7,8.50,0,0.5,72,1.01,44.2984,393.7,324,102,20,NaN,6109,18.9474


#Поиск пропусков

In [ ]:
data.isna()

,Depth_m,Litology,Bit_load_tonnes,Rotor_speed,Inlet_density_g/cc,Inlet_flow_litres/sec,D_bit_mm,D_well_mm,Complication_code_mm,Stratigraphy_code,Markers_horizon_code,Oil_well,Aver_mech_speed_m/h
0,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36111,False,False,False,False,False,False,False,False,True,False,False,False,False
36112,False,False,False,False,False,False,False,False,True,False,False,False,False
36113,False,False,False,False,False,False,False,False,True,False,False,False,False
36114,False,False,False,False,False,False,False,False,True,False,False,False,False


#Удаление пропусков

In [ ]:
df_list=data.dropna(axis=1)

In [ ]:
df_list

,Depth_m,Litology,Bit_load_tonnes,Rotor_speed,Inlet_density_g/cc,Inlet_flow_litres/sec,D_bit_mm,D_well_mm,Stratigraphy_code,Oil_well,Aver_mech_speed_m/h
0,7.80,0,0.000,56,1.01,43.2682,393.7,324,20,6109,0.0690
1,7.90,0,0.000,56,1.01,42.9248,393.7,324,20,6109,4.7368
2,8.01,0,0.000,72,1.01,43.0965,393.7,324,20,6109,2.4444
3,8.11,0,0.000,72,1.01,43.0965,393.7,324,20,6109,7.3469
4,8.20,0,0.000,72,1.01,42.9248,393.7,324,20,6109,7.7143
...,...,...,...,...,...,...,...,...,...,...,...
36111,3753.71,10,7.550,111,1.00,14.0816,155.6,114,622,6022,32.7273
36112,3753.80,10,7.294,111,1.00,14.0816,155.6,114,622,6022,24.9231
36113,3753.93,10,6.911,111,1.00,14.1381,155.6,114,622,6022,22.2857
36114,3754.01,10,5.904,111,1.00,14.1381,155.6,114,622,6022,16.9412


In [ ]:
df_list_1=df_list.dropna(axis=0)

In [ ]:
df_list_1

,Depth_m,Litology,Bit_load_tonnes,Rotor_speed,Inlet_density_g/cc,Inlet_flow_litres/sec,D_bit_mm,D_well_mm,Stratigraphy_code,Oil_well,Aver_mech_speed_m/h
0,7.80,0,0.000,56,1.01,43.2682,393.7,324,20,6109,0.0690
1,7.90,0,0.000,56,1.01,42.9248,393.7,324,20,6109,4.7368
2,8.01,0,0.000,72,1.01,43.0965,393.7,324,20,6109,2.4444
3,8.11,0,0.000,72,1.01,43.0965,393.7,324,20,6109,7.3469
4,8.20,0,0.000,72,1.01,42.9248,393.7,324,20,6109,7.7143
...,...,...,...,...,...,...,...,...,...,...,...
36111,3753.71,10,7.550,111,1.00,14.0816,155.6,114,622,6022,32.7273
36112,3753.80,10,7.294,111,1.00,14.0816,155.6,114,622,6022,24.9231
36113,3753.93,10,6.911,111,1.00,14.1381,155.6,114,622,6022,22.2857
36114,3754.01,10,5.904,111,1.00,14.1381,155.6,114,622,6022,16.9412


# Корреляция

In [ ]:
corr_df_list = df_list_1.corr()
corr_df_list.style.background_gradient(cmap='coolwarm')

,Depth_m,Litology,Bit_load_tonnes,Rotor_speed,Inlet_density_g/cc,Inlet_flow_litres/sec,D_bit_mm,D_well_mm,Stratigraphy_code,Oil_well,Aver_mech_speed_m/h
Depth_m,1.000000,0.137832,-0.211166,0.608141,-0.401161,-0.841156,-0.892511,-0.899551,0.760894,0.042696,-0.148990
Litology,0.137832,1.000000,-0.184594,0.185408,-0.179854,-0.251430,-0.192696,-0.196870,0.233275,-0.043873,-0.080922
Bit_load_tonnes,-0.211166,-0.184594,1.000000,-0.349506,0.376555,0.360208,0.246572,0.265363,-0.349112,0.251603,0.080786
Rotor_speed,0.608141,0.185408,-0.349506,1.000000,-0.548860,-0.611355,-0.568612,-0.590285,0.605735,-0.032929,0.257608
Inlet_density_g/cc,-0.401161,-0.179854,0.376555,-0.548860,1.000000,0.494463,0.408053,0.437953,-0.503170,0.094986,-0.147164
Inlet_flow_litres/sec,-0.841156,-0.251430,0.360208,-0.611355,0.494463,1.000000,0.933830,0.942373,-0.830423,0.129332,0.199645
D_bit_mm,-0.892511,-0.192696,0.246572,-0.568612,0.408053,0.933830,1.000000,0.997227,-0.798685,0.021225,0.180063
D_well_mm,-0.899551,-0.196870,0.265363,-0.590285,0.437953,0.942373,0.997227,1.000000,-0.814114,0.023405,0.172944
Stratigraphy_code,0.760894,0.233275,-0.349112,0.605735,-0.503170,-0.830423,-0.798685,-0.814114,1.000000,-0.041365,-0.139825
Oil_well,0.042696,-0.043873,0.251603,-0.032929,0.094986,0.129332,0.021225,0.023405,-0.041365,1.000000,0.192663


#Убираем коррелирующие признаки

In [ ]:
df_list_1=df_list_1.drop(['D_bit_mm','D_well_mm','Inlet_flow_litres/sec'], axis = 1)

In [ ]:
df_list_1

,Depth_m,Litology,Bit_load_tonnes,Rotor_speed,Inlet_density_g/cc,Stratigraphy_code,Oil_well,Aver_mech_speed_m/h
0,7.80,0,0.000,56,1.01,20,6109,0.0690
1,7.90,0,0.000,56,1.01,20,6109,4.7368
2,8.01,0,0.000,72,1.01,20,6109,2.4444
3,8.11,0,0.000,72,1.01,20,6109,7.3469
4,8.20,0,0.000,72,1.01,20,6109,7.7143
...,...,...,...,...,...,...,...,...
36111,3753.71,10,7.550,111,1.00,622,6022,32.7273
36112,3753.80,10,7.294,111,1.00,622,6022,24.9231
36113,3753.93,10,6.911,111,1.00,622,6022,22.2857
36114,3754.01,10,5.904,111,1.00,622,6022,16.9412


In [ ]:
df_list_1.isnull().sum()

Depth_m                0
Litology               0
Bit_load_tonnes        0
Rotor_speed            0
Inlet_density_g/cc     0
Stratigraphy_code      0
Oil_well               0
Aver_mech_speed_m/h    0
dtype: int64

In [ ]:
df_list_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174830 entries, 0 to 36115
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Depth_m              174830 non-null  float64
 1   Litology             174830 non-null  int64  
 2   Bit_load_tonnes      174830 non-null  float64
 3   Rotor_speed          174830 non-null  int64  
 4   Inlet_density_g/cc   174830 non-null  float64
 5   Stratigraphy_code    174830 non-null  int64  
 6   Oil_well             174830 non-null  int64  
 7   Aver_mech_speed_m/h  174830 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 12.0 MB


In [ ]:
df_list_1.keys()

Index(['Depth_m', 'Litology', 'Bit_load_tonnes', 'Rotor_speed',
       'Inlet_density_g/cc', 'Stratigraphy_code', 'Oil_well',
       'Aver_mech_speed_m/h'],
      dtype='object')

#Popularity-based подход

In [ ]:
df_price = df_list_1.groupby(by=['Stratigraphy_code', 'Depth_m', 'Oil_well'], as_index=False).agg({'Aver_mech_speed_m/h':['max'] })
df_price

,Stratigraphy_code,Depth_m,Oil_well,Aver_mech_speed_m/h
,,,,max
0,10,3.10,6010,60.0000
1,10,11.10,6201,15.6522
2,10,11.21,6201,23.2941
3,10,11.30,6201,10.1250
4,10,11.40,6201,18.9474
...,...,...,...,...
174825,622,3996.70,6200,72.0000
174826,622,3996.80,6200,387.0000
174827,622,3996.91,6200,44.0000


In [ ]:
a=df_price['Stratigraphy_code']
e=df_price['Depth_m']
b=df_price['Oil_well']

d=df_price['Aver_mech_speed_m/h']['max']

df = pd.concat([a,b, e, d], axis=1)
df

,Stratigraphy_code,Oil_well,Depth_m,max
0,10,6010,3.10,60.0000
1,10,6201,11.10,15.6522
2,10,6201,11.21,23.2941
3,10,6201,11.30,10.1250
4,10,6201,11.40,18.9474
...,...,...,...,...
174825,622,6200,3996.70,72.0000
174826,622,6200,3996.80,387.0000
174827,622,6200,3996.91,44.0000
174828,622,6200,3997.12,86.4000


In [ ]:
df.nlargest(5, ['max'])

,Stratigraphy_code,Oil_well,Depth_m,max
99574,361,6200,1227.12,612.0
49530,39,6201,1606.34,504.0
85084,360,6201,1387.94,504.0
6742,32,6201,164.70,468.0
50679,39,6010,1629.77,468.0


#Content-based подход

#Пробуренные скважины

#Загрузка данных

In [ ]:
da_6109 = pd.read_csv('/content/drive/MyDrive/ML/DZ/Drilling_error_prediction/DATA/6109 выгрузка по глубине+траппы+стратигр.csv', index_col=0)
db_6200 = pd.read_csv('/content/drive/MyDrive/ML/DZ/Drilling_error_prediction/DATA/6200 выгрузка по глубине+траппы+стратигр.csv', index_col=0)
dc_6010 = pd.read_csv('/content/drive/MyDrive/ML/DZ/Drilling_error_prediction/DATA/Выгрузка 6010 глубина+страт+траппы.csv', index_col=0)
dd_6022 = pd.read_csv('/content/drive/MyDrive/ML/DZ/Drilling_error_prediction/DATA/Выгрузка 6022 глубина+страт+траппы.csv', index_col=0)
de_6201 = pd.read_csv('/content/drive/MyDrive/ML/DZ/Drilling_error_prediction/DATA/6201 выгрузка по глубине+траппы+стратигр.csv', index_col=0)

#Предобработка данных

In [ ]:
da_1 = da_6109.rename(columns={'Глубина': 'Depth_m_6109', 'Код стратиграфии': 'Stratigraphy_code'})
db_1 = db_6200.rename(columns={'Глубина': 'Depth_m_6200', 'Код стратиграфии': 'Stratigraphy_code'})
dc_1 = dc_6010.rename(columns={'Глубина': 'Depth_m_6010', 'Код стратиграфии': 'Stratigraphy_code'})
dd_1 = dd_6022.rename(columns={'Глубина': 'Depth_m_6022', 'Код стратиграфии': 'Stratigraphy_code'})
de_1 = de_6201.rename(columns={'Глубина': 'Depth_m_6201', 'Код стратиграфии': 'Stratigraphy_code'})
da_6109 =da_1[['Depth_m_6109', 'Stratigraphy_code']]
db_6200 =db_1[['Depth_m_6200', 'Stratigraphy_code']]
dc_6010 =dc_1[['Depth_m_6010', 'Stratigraphy_code']]
dd_6022 =dd_1[['Depth_m_6022', 'Stratigraphy_code']]
de_6201 =de_1[['Depth_m_6201', 'Stratigraphy_code']]

#Усреднение данных и приведение к одной размерности

In [ ]:
da_6109=da_6109.iloc[::100]
da_6109=da_6109[:312]
db_6200=db_6200.iloc[::100]
db_6200=db_6200[:312]
dc_6010=dc_6010.iloc[::100]
dc_6010=dc_6010[:312]
dd_6022=dd_6022.iloc[::100]
dd_6022=dd_6022[:312]
de_6201=de_6201.iloc[::100]
de_6201=de_6201[:312]

#Пропуски и векторизация

In [ ]:
da_6109=da_6109.dropna(axis=0)
da_6109_ar = da_6109.to_numpy()
db_6200 =db_6200 .dropna(axis=0)
db_6200_ar = db_6200.to_numpy()
dc_6010=dc_6010.dropna(axis=0)
dc_6010_ar = dc_6010.to_numpy()
dd_6022=dd_6022.dropna(axis=0)
dd_6022_ar = dd_6022.to_numpy()
de_6201 =de_6201.dropna(axis=0)
de_6201_ar = de_6201.to_numpy()

# Новая не пробуренная скважина

In [ ]:
data_1 = pd.read_excel("/content/drive/MyDrive/ML/DZ/Drilling_error_prediction/8302_ver_new_well.xlsx", 
                 #encoding = 'ISO-8859-1', 
                 index_col=0)
data_1.head()

,Глубина,Код Литологии,Диаметр долота,Диаметр скважины,Код осложнений,Код стратиграфии
id,,,,,,
1,11.6,10,0.3937,0.324,0,20
2,11.7,10,0.3937,0.324,0,20
3,11.8,10,0.3937,0.324,0,20
4,11.9,10,0.3937,0.324,0,20
5,12.0,10,0.3937,0.324,0,20


In [ ]:
df_1 = data_1.rename(columns={'Глубина': 'Depth_m_8302', 'Код стратиграфии': 'Stratigraphy_code'})
df_8302 =df_1[['Depth_m_8302', 'Stratigraphy_code']]
# df_1.insert(1, "Oil_well", 8302)
df_8302

,Depth_m_8302,Stratigraphy_code
id,,
1,11.60,20
2,11.70,20
3,11.80,20
4,11.90,20
5,12.00,20
...,...,...
31296,3152.90,622
31297,3153.01,622
31298,3153.11,622


In [ ]:
df_8302=df_8302.iloc[::100]
df_8302=df_8302[:312]
df_8302 

,Depth_m_8302,Stratigraphy_code
id,,
1,11.6,20
101,21.6,20
201,31.6,30
301,45.9,30
401,55.9,30
...,...,...
30701,3093.4,622
30801,3103.4,622
30901,3113.4,622


In [ ]:
df_8302=df_8302.dropna(axis=0)
df_8302_ar = df_8302.to_numpy()
df_8302_ar

In [ ]:
df_8302.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312 entries, 1 to 31101
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Depth_m_8302       312 non-null    float64
 1   Stratigraphy_code  312 non-null    int64  
dtypes: float64(1), int64(1)
memory usage: 7.3 KB


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

r=cosine_similarity(de_6201_ar.reshape(1,-1),df_8302_ar.reshape(1,-1))
r1=cosine_similarity(dd_6022_ar.reshape(1,-1),df_8302_ar.reshape(1,-1))
r2=cosine_similarity(dc_6010_ar.reshape(1,-1),df_8302_ar.reshape(1,-1))
r3=cosine_similarity(da_6109_ar.reshape(1,-1),df_8302_ar.reshape(1,-1))
r4=cosine_similarity(db_6200_ar.reshape(1,-1),df_8302_ar.reshape(1,-1))
a=max(r,r1,r2,r3,r4)
print(a)
print(r,r1,r2,r3,r4)


[[0.99931335]]
[[0.99615529]] [[0.99931335]] [[0.99916434]] [[0.99719656]] [[0.99397241]]


#Наиболее похожая скважина №6022 и ее параметры

In [ ]:
dd_6022 = pd.read_csv('/content/drive/MyDrive/ML/DZ/Drilling_error_prediction/DATA/Выгрузка 6022 глубина+страт+траппы.csv', index_col=0)
dd_6022

,Глубина,Код Литологии,С1,С2,С3,iС4,С4,IC5,C5,Вес на кр,Нагр. На долото,Давление,Обороты рот,Положение кр,Положение долота,Мех. скорость ср,Плотность на вх,Плотность на вых,Температура вх,Температура вых,Расход на вх,Расход на вых.,Рабочая емк. 1,Рабочая емк. 2,ЦСГО 1,ЦСГО 2,Долив,БПР 1,Сумма актив. емкостей,Диаметр долота,Диаметр скважины,Код осложнений,Код стратиграфии,Код горизонта маркера
1,10.30,8.0,0.0017,0.0028,0.0001,0.0000,0.0001,0.0000,0.0000,13.147,0.083,3.6,18.0,9.85,10.30,5.4340,-999.25,1.23,-999.25,20.99,20.9334,72.5167,10.296,-0.9992,6.504,3.942,-0.10,11.042,21.488,393.7,324.0,NaN,20.0,NaN
2,10.41,8.0,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,13.139,0.091,3.6,18.0,9.73,10.41,7.6154,-999.25,-999.25,-999.25,-999.25,20.9334,72.5167,10.301,-0.9992,7.387,3.951,-0.10,9.743,21.081,393.7,324.0,NaN,20.0,NaN
3,10.50,8.0,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,13.176,0.054,3.6,18.0,9.62,10.50,6.7500,-999.25,-999.25,-999.25,-999.25,20.9334,72.5167,10.344,-0.9992,8.134,3.933,-0.10,8.412,20.479,393.7,324.0,NaN,20.0,NaN
4,10.60,8.0,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,13.123,0.107,4.5,18.0,9.51,10.60,7.3469,-999.25,-999.25,-999.25,-999.25,21.6647,75.0500,10.414,-0.9992,7.913,4.445,-0.10,7.082,19.440,393.7,324.0,NaN,20.0,NaN
5,10.70,8.0,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,13.231,0.000,4.5,18.0,9.39,10.70,5.9016,-999.25,-999.25,-999.25,-999.25,22.7616,78.8500,10.447,-0.9992,8.098,5.199,-0.11,7.535,20.832,393.7,324.0,NaN,20.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37183,3753.71,10.0,0.0171,0.0029,0.0007,0.0006,0.0007,0.0006,0.0007,47.186,7.550,169.2,111.0,0.85,3753.71,32.7273,1.00,1.01,21.05,26.05,14.0816,91.3000,36.971,-0.9992,7.494,8.483,3.14,15.270,56.088,155.6,114.0,NaN,622.0,100.0
37184,3753.80,10.0,0.0171,0.0029,0.0007,0.0006,0.0007,0.0006,0.0007,47.442,7.294,169.2,111.0,0.76,3753.80,24.9231,1.00,1.01,21.05,26.05,14.0816,91.3000,36.954,-0.9992,7.492,8.505,3.14,15.255,56.091,155.6,114.0,NaN,622.0,100.0
37185,3753.93,10.0,0.0171,0.0029,0.0007,0.0006,0.0007,0.0006,0.0007,47.825,6.911,170.1,111.0,0.63,3753.93,22.2857,1.00,1.01,21.05,26.05,14.1381,91.6667,36.954,-0.9992,7.494,8.534,3.14,15.263,56.122,155.6,114.0,NaN,622.0,100.0
37186,3754.01,10.0,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,-999.2500,48.832,5.904,169.2,111.0,0.55,3754.01,16.9412,1.00,-999.25,21.05,-999.25,14.1381,91.6667,36.886,-0.9992,7.514,8.488,3.14,15.271,56.028,155.6,114.0,NaN,622.0,100.0


# Вывод: 
1. Popularity-based подход дает нам рекомендации по выбору максимальной скорости для бурения новой скважины в зависимости от стратиграфии.
2. Content-based подход дает ответ среди пробуренных скважин, которая наиболее подходит по своим параметрам для бурения новой скважины.